## Imports

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
projectdir = "/home/jovyan/work/MED_Fall"
workdir = "/home/jovyan/work"

In [3]:
import os

os.environ["PYTHONPATH"]

'/home/jovyan/work/MED_Fall/'

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread

from vision.pre_processing.extract_frames import FramesExtractor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from utils.utility_functions import listdir_nohidden_sorted as lsdir
from utils.utility_functions import load_images, show_images
import glob

In [5]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Setup the dataframe for keras flow_from_dataframe pipeline

In [6]:
extracted_frames_path = (
    "/home/jovyan/work/MED_Fall/vision/vision_dataset/extracted_frames"
)

In [7]:
ground_truth_old_path = (
    "/home/jovyan/work/MED_Fall/vision/vision_dataset/ground_truth_old"
)
ground_truth_path = "/home/jovyan/work/MED_Fall/vision/vision_dataset/ground_truth"
len(lsdir(ground_truth_path))

50

In [8]:
dfs = []
for file in lsdir(ground_truth_path):
    df = pd.read_csv(file)
    dfs.append(df)
dataset = pd.concat(dfs, ignore_index=True)

In [9]:
dfs_old = []
for file in lsdir(ground_truth_old_path):
    df_old = pd.read_csv(file)
    dfs_old.append(df_old)
dataset_old = pd.concat(dfs_old, ignore_index=True)

In [10]:
dataset["frame_name"] = (
    dataset_old["frame_name"] + ".jpg"
)  # add extension to file_names

dataset = dataset.iloc[:, 1:]
dataset

,micro_labels,macro_labels,ar_labels,frame_name
0,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0000.jpg
1,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0001.jpg
2,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0002.jpg
3,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0003.jpg
4,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0004.jpg
...,...,...,...,...
1182295,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4615.jpg
1182296,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4616.jpg
1182297,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4617.jpg
1182298,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4618.jpg


### Visualize some frames from the dataset

In [12]:
##removector actor repositioning frames
on_air_ds = dataset.loc[dataset["ar_labels"] == "on_air"]
on_air_ds

,micro_labels,macro_labels,ar_labels,frame_name
206,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0206.jpg
207,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0207.jpg
208,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0208.jpg
209,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0209.jpg
210,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0210.jpg
...,...,...,...,...
1182213,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4533.jpg
1182214,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4534.jpg
1182215,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4535.jpg
1182216,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4536.jpg


In [13]:
rand_samples = on_air_ds.groupby("macro_labels").sample(n=10)
rand_samples.head()

,micro_labels,macro_labels,ar_labels,frame_name
729793,stand_up_from_sit,adl,on_air,actor_2_walk_armour_obj_cam_1_2353.jpg
485144,sit_up_from_lying,adl,on_air,actor_2_bed_armour_cam_5_3104.jpg
431421,sit_up_from_lying,adl,on_air,actor_2_bed_cam_2_1701.jpg
892584,stand_still,adl,on_air,actor_3_chair_full_ph_cam_6_0264.jpg
10256,sit_up_from_lying,adl,on_air,actor_1_bed_cam_3_1616.jpg


In [14]:
##concatenare alla macro label il nome del fotogramma
titles = rand_samples["frame_name"] + " -- " + rand_samples["macro_labels"]
titles

729793         actor_2_walk_armour_obj_cam_1_2353.jpg -- adl
485144              actor_2_bed_armour_cam_5_3104.jpg -- adl
431421                     actor_2_bed_cam_2_1701.jpg -- adl
892584           actor_3_chair_full_ph_cam_6_0264.jpg -- adl
10256                      actor_1_bed_cam_3_1616.jpg -- adl
239445           actor_1_walk2_full_ph_cam_4_3945.jpg -- adl
993491      actor_3_walk_stick_full_ph_cam_4_3131.jpg -- adl
212448           actor_1_walk2_full_ph_cam_1_0348.jpg -- adl
62133              actor_1_bed_full_ph_cam_7_2373.jpg -- adl
739531         actor_2_walk_armour_obj_cam_5_1051.jpg -- adl
169496       actor_1_chair_full_ph_cam_4_2936.jpg -- falling
21263                  actor_1_bed_cam_5_3983.jpg -- falling
101319         actor_1_bed_rolling_cam_7_0279.jpg -- falling
649316     actor_2_rolling_full_ph_cam_2_1136.jpg -- falling
363683             actor_1_walk_ph_cam_6_4943.jpg -- falling
285048                actor_1_walk_cam_4_1068.jpg -- falling
903891             actor

In [ ]:
dataset.loc[dataset["frame_name"] == "actor_2_bed_cam_3_3843.jpg"]

In [ ]:
images = load_images(extracted_frames_path, rand_samples["frame_name"])

show_images(
    images,
    rows=5,
    figsize=(30, 20),
    titles=titles,
)

## Subsample dataset choosing Actor 4 as fine tuning data

In [15]:
# select only actor 4
actor_4_dataset = dataset[dataset["frame_name"].str.contains("actor_4")]

In [16]:
actor_4_dataset.reset_index(drop=True, inplace=True)
actor_4_dataset

,micro_labels,macro_labels,ar_labels,frame_name
0,lie_still,lying_down,actor_repositioning,actor_4_bed_cam_1_0000.jpg
1,lie_still,lying_down,actor_repositioning,actor_4_bed_cam_1_0001.jpg
2,lie_still,lying_down,actor_repositioning,actor_4_bed_cam_1_0002.jpg
3,lie_still,lying_down,actor_repositioning,actor_4_bed_cam_1_0003.jpg
4,lie_still,lying_down,actor_repositioning,actor_4_bed_cam_1_0004.jpg
...,...,...,...,...
168415,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4615.jpg
168416,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4616.jpg
168417,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4617.jpg
168418,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4618.jpg


In [17]:
actor_4_dataset = actor_4_dataset.loc[actor_4_dataset["ar_labels"] == "on_air"]
actor_4_dataset

,micro_labels,macro_labels,ar_labels,frame_name
126,sit_up_from_lying,adl,on_air,actor_4_bed_cam_1_0126.jpg
127,sit_up_from_lying,adl,on_air,actor_4_bed_cam_1_0127.jpg
128,sit_up_from_lying,adl,on_air,actor_4_bed_cam_1_0128.jpg
129,sit_up_from_lying,adl,on_air,actor_4_bed_cam_1_0129.jpg
130,sit_up_from_lying,adl,on_air,actor_4_bed_cam_1_0130.jpg
...,...,...,...,...
168333,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4533.jpg
168334,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4534.jpg
168335,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4535.jpg
168336,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4536.jpg


In [18]:
some_actor_4_samples = actor_4_dataset.groupby("macro_labels").sample(n=5)
images = load_images(extracted_frames_path, some_actor_4_samples["frame_name"])

show_images(images, titles=some_actor_4_samples["macro_labels"])

TypeError: load_images() missing 1 required positional argument: 'resize_shape'

In [19]:
train_set = actor_4_dataset.sample(frac=0.9, random_state=2)
val_set = actor_4_dataset.drop(train_set.index)

print(train_set.shape, val_set.shape)

(35431, 4) (3937, 4)


In [20]:
train_set["macro_labels"].value_counts()

adl           23140
falling       11677
lying_down      614
Name: macro_labels, dtype: int64

In [21]:
adl_train = train_set.loc[train_set["macro_labels"] == "adl"]
adl_train.reset_index(drop=True, inplace=True)
adl_train

,micro_labels,macro_labels,ar_labels,frame_name
0,stand_up_from_sit,adl,on_air,actor_4_chair_cam_3_1280.jpg
1,rolling_bed,adl,on_air,actor_4_bed_full_ph_cam_3_6665.jpg
2,rolling_bed,adl,on_air,actor_4_bed_full_ph_cam_7_6653.jpg
3,sit_up_from_lying,adl,on_air,actor_4_bed_full_ph_cam_7_0827.jpg
4,sit_up_from_lying,adl,on_air,actor_4_bed_full_ph_cam_5_2467.jpg
...,...,...,...,...
23135,sit_up_from_lying,adl,on_air,actor_4_bed_full_ph_cam_7_0718.jpg
23136,sit_up_from_lying,adl,on_air,actor_4_bed_cam_1_0594.jpg
23137,sit_up_from_lying,adl,on_air,actor_4_bed_full_ph_cam_4_3192.jpg
23138,stand_up_from_sit,adl,on_air,actor_4_chair_full_ph_cam_3_1521.jpg


In [22]:
lying_down_n_samples = train_set["macro_labels"].value_counts()[1]
lying_down_n_samples

11677

In [23]:
adl_train_under = adl_train.sample(n=lying_down_n_samples, random_state=2)
adl_train_under

,micro_labels,macro_labels,ar_labels,frame_name
15847,sit_up_from_lying,adl,on_air,actor_4_bed_full_ph_cam_2_3172.jpg
10831,sit_up_from_lying,adl,on_air,actor_4_bed_full_ph_cam_1_1795.jpg
21927,stand_up_from_sit,adl,on_air,actor_4_chair_cam_4_1319.jpg
2059,stand_up_from_sit,adl,on_air,actor_4_chair_full_ph_cam_1_4425.jpg
11744,sit_up_from_lying,adl,on_air,actor_4_bed_full_ph_cam_5_0807.jpg
...,...,...,...,...
9346,stand_up_from_sit,adl,on_air,actor_4_chair_cam_7_1280.jpg
1622,stand_up_from_sit,adl,on_air,actor_4_bed_cam_1_1225.jpg
22070,sit_up_from_lying,adl,on_air,actor_4_bed_cam_5_2274.jpg
2725,sit_up_from_lying,adl,on_air,actor_4_bed_full_ph_cam_4_0724.jpg


In [24]:
train_set_copy = train_set.copy()
train_set_copy = train_set_copy[train_set_copy["macro_labels"] != "adl"]
train_set_copy

,micro_labels,macro_labels,ar_labels,frame_name
158570,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_5_4010.jpg
72172,lie_down_on_the_floor,lying_down,on_air,actor_4_bed_full_ph_cam_5_5812.jpg
156754,fall_lateral,falling,on_air,actor_4_chair_full_ph_cam_5_2194.jpg
111137,crouched_still,falling,on_air,actor_4_bed_full_ph_knee_cam_7_1217.jpg
18422,lie_down_on_the_floor,lying_down,on_air,actor_4_bed_cam_7_1862.jpg
...,...,...,...,...
146953,fall_lateral,falling,on_air,actor_4_chair_full_ph_cam_3_1633.jpg
17235,fall_frontal,falling,on_air,actor_4_bed_cam_7_0675.jpg
142899,fall_lateral,falling,on_air,actor_4_chair_full_ph_cam_2_2199.jpg
17262,fall_frontal,falling,on_air,actor_4_bed_cam_7_0702.jpg


In [25]:
train_set_copy["macro_labels"].value_counts()

falling       11677
lying_down      614
Name: macro_labels, dtype: int64

In [26]:
train_set_copy = pd.concat([train_set_copy, adl_train_under], axis=0)
train_set_copy

,micro_labels,macro_labels,ar_labels,frame_name
158570,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_5_4010.jpg
72172,lie_down_on_the_floor,lying_down,on_air,actor_4_bed_full_ph_cam_5_5812.jpg
156754,fall_lateral,falling,on_air,actor_4_chair_full_ph_cam_5_2194.jpg
111137,crouched_still,falling,on_air,actor_4_bed_full_ph_knee_cam_7_1217.jpg
18422,lie_down_on_the_floor,lying_down,on_air,actor_4_bed_cam_7_1862.jpg
...,...,...,...,...
9346,stand_up_from_sit,adl,on_air,actor_4_chair_cam_7_1280.jpg
1622,stand_up_from_sit,adl,on_air,actor_4_bed_cam_1_1225.jpg
22070,sit_up_from_lying,adl,on_air,actor_4_bed_cam_5_2274.jpg
2725,sit_up_from_lying,adl,on_air,actor_4_bed_full_ph_cam_4_0724.jpg


In [27]:
train_set_copy["macro_labels"].value_counts()

falling       11677
adl           11677
lying_down      614
Name: macro_labels, dtype: int64

In [28]:
some_samples = train_set_copy.groupby("macro_labels").sample(n=5)
images = load_images(extracted_frames_path, some_samples["frame_name"])

show_images(images, titles=some_samples["macro_labels"])

TypeError: load_images() missing 1 required positional argument: 'resize_shape'

## Fine tune VGG16 on Actor 4

### Creating Training and Validation Generators

In [29]:
train_generator = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


train_datagen = train_generator.flow_from_dataframe(
    train_set_copy,
    directory=extracted_frames_path,
    x_col="frame_name",
    y_col="macro_labels",
    target_size=(224, 224),
    color_mode="rgb",
    classes=["adl", "lying_down", "falling"],
    class_mode="categorical",
    batch_size=64,
    shuffle=True,
    seed=2,
)

Found 23968 validated image filenames belonging to 3 classes.


In [30]:
val_generator = ImageDataGenerator(rescale=1.0 / 255)

val_datagen = val_generator.flow_from_dataframe(
    val_set,
    directory=extracted_frames_path,
    x_col="frame_name",
    y_col="macro_labels",
    target_size=(224, 224),
    color_mode="rgb",
    classes=["adl", "lying_down", "falling"],
    class_mode="categorical",
    batch_size=64,
    shuffle=False,
    seed=2,
)

Found 3937 validated image filenames belonging to 3 classes.


In [31]:
print(pd.Series(train_datagen.classes).value_counts())
print(train_datagen.class_indices)

1    11677
0    11677
2      614
dtype: int64
{'adl': 0, 'falling': 1, 'lying_down': 2}


In [32]:
print(pd.Series(val_datagen.classes).value_counts())
print(val_datagen.class_indices)

0    2564
1    1315
2      58
dtype: int64
{'adl': 0, 'falling': 1, 'lying_down': 2}


### Download pre-trained VGG16

In [33]:
IMG_SIZE = (224, 224, 3)

In [34]:
feature_extractor = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=IMG_SIZE,
    pooling="avg",
)

feature_extractor.summary()

2022-08-22 13:36:36.882110: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-22 13:36:37.304866: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21353 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:65:00.0, compute capability: 8.6


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### Freeze all layers except last convolutional block

In [35]:
# freeze all layers except last five
for layer in feature_extractor.layers[:-9]:
    layer.trainable = False

for i, layer in enumerate(feature_extractor.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 True
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 global_average_pooling2d True


### Add top layer to fine tune

In [36]:
x = feature_extractor.output
x = Flatten()(x)
x = Dense(units=512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(units=256, activation="relu")(x)
x = Dense(units=3, activation="softmax")(x)

transfer_model = Model(inputs=feature_extractor.input, outputs=x)

transfer_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### Compile the model and define callbacks

In [37]:
transfer_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.Recall(class_id=0),
        tf.keras.metrics.Recall(class_id=1),
        tf.keras.metrics.Recall(class_id=2),
    ],
)

In [38]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f"{projectdir}/vision/model_checkpoints/resample/vgg_top_fine_tuned_best_epoch_undersample.h5",
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    save_freq="epoch",
)

In [39]:
callbacks = [
    # model_checkpoint,
    # tf.keras.callbacks.ReduceLROnPlateau(verbose=1),
    tf.keras.callbacks.EarlyStopping(patience=3, verbose=1),
]

### Train the model

In [40]:
history = transfer_model.fit(
    train_datagen, validation_data=val_datagen, epochs=20, callbacks=callbacks
)

2022-08-22 13:36:46.723833: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-08-22 13:36:48.489279: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-08-22 13:36:52.191892: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  logits and labels must have the same first dimension, got logits shape [64,3] and labels shape [192]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at tmp/ipykernel_358705/607337168.py:1) ]]
	 [[assert_less_equal/Assert/AssertGuard/pivot_f/_13/_43]]
  (1) Invalid argument:  logits and labels must have the same first dimension, got logits shape [64,3] and labels shape [192]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at tmp/ipykernel_358705/607337168.py:1) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_2304]

Function call stack:
train_function -> train_function


### Evaluate the model

In [ ]:
##load best checkpoint
best_model = tf.keras.models.load_model(
    f"{projectdir}/vision/model_checkpoints/resample/vgg_top_fine_tuned_best_epoch_undersample.h5"
)

In [ ]:
best_model.evaluate(val_datagen)

In [ ]:
plt.figure()
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "val"], loc="upper left")
plt.grid()
plt.show()

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "val"], loc="upper left")
plt.grid()
plt.show()

In [ ]:
y_preds_logits = best_model.predict(val_datagen)

In [ ]:
y_preds = np.argmax(y_preds_logits, axis=1)

In [ ]:
print(
    classification_report(
        y_true=val_datagen.classes,
        y_pred=y_preds,
        target_names=list(val_datagen.class_indices.keys()),
    )
)

In [ ]:
pd.Series(train_datagen.classes).value_counts()

In [ ]:
#%cp ~/work/MED_Fall/vision/model_checkpoints/resample/vgg_top_fine_tuned_best_epoch_undersample.h5 ~/work/persistent/

In [ ]:
#load_images()

## Save Features Extractor

In [ ]:
best_model.summary()

In [ ]:
feature_extractor = Sequential(best_model.layers[:-5])
feature_extractor.summary()

In [ ]:
for layer in feature_extractor.layers:
    layer.trainable = False
for layer in feature_extractor.layers:
    print(layer.trainable)

In [ ]:
#feature_extractor.save(f"{projectdir}/vision/models/vgg_feature_extractor.h5")

In [ ]:
pred_test = feature_extractor.predict(np.ones(shape=(1, 224, 224, 3)))
print(pred_test.shape)